In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [2]:

# Set image size and paths
img_width, img_height = 224, 224
train_dir = r"D:\Deep learning\Panda_bear_data_set\PandasBears\Test"
val_dir = r"D:\Deep learning\Panda_bear_data_set\PandasBears\Train"
num_classes = 2  # Change this to 10 if you have 10 dog breeds

In [3]:
# Data preprocessing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical' 
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical' 
)

Found 100 images belonging to 2 classes.


Found 500 images belonging to 2 classes.


In [5]:

# Load pre-trained VGG16 without the top classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

In [6]:

# Add custom top layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(num_classes, activation='softmax' if num_classes > 2 else 'sigmoid')(x)

In [7]:
# Build full model
model = Model(inputs=base_model.input, outputs=output_layer)
model.summary()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)


Epoch 1/10
4/4 [==============================] - 69s 17s/step - loss: 4.6985 - accuracy: 0.8400 - val_loss: 0.2301 - val_accuracy: 0.9940
Epoch 2/10
4/4 [==============================] - 14s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.1275 - val_accuracy: 0.9980
Epoch 3/10
4/4 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.1043 - val_accuracy: 0.9980
Epoch 4/10
4/4 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0860 - val_accuracy: 0.9980
Epoch 5/10
4/4 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0727 - val_accuracy: 0.9980
Epoch 6/10
4/4 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0642 - val_accuracy: 0.9980
Epoch 7/10
4/4 [==============================] - 13s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0588 - val_accuracy: 0.9980
Epoch 8/1

In [9]:
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

16/16 [==============================] - 11s 671ms/step - loss: 0.0507 - accuracy: 0.9980
Validation Accuracy: 99.80%
